This main notebook is to connect the SciAnalysis to the perovskite work we are doing

In [1]:
# cloning the SciAnalysis repo for immediate use
# git clone https://github.com/CFN-softbio/SciAnalysis.git

# for users that has this already install, please just reference the repo